#Install Libraries

In [2]:
!pip install -U datasets transformers accelerate pandas pyarrow chardet nltk spacy evaluate

  Using cached transformers-4.55.4-py3-none-any.whl.metadata (41 kB)
Using cached transformers-4.55.4-py3-none-any.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [13]:
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 transformers==4.44.2 evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 838.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 877.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 852.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.

#Import Libraries

In [1]:
import torch
import re,string
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from torch.utils.data import DataLoader
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

Torch: 2.8.0+cu128
CUDA available: True


#Load Dataset

In [3]:
#From huggingface
dataset = load_dataset('imdb')
print(dataset)

#Custom
#dataset = load_dataset("csv",data_files={'data':'custom.csv'})['data']
#print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


#Clean Data

In [4]:
re_url=True
re_html=True
re_stopwords=False
lower=True
re_punc=True

urls = re.compile(r"(www\.\S+|https?://\S+)")
htmls = re.compile(r"(<.*?>)")
stopwords = set(stopwords.words("english")) if re_stopwords else set()
punct = str.maketrans("","",string.punctuation)

In [5]:
def clean_text(x):
  if re_url:
    x=urls.sub(" ",x)
  if re_html:
    x=htmls.sub(" ",x)
  if lower:
    x.lower()
  if re_stopwords:
    x= " ".join([w for w in x.split() if w not in stopwords])
  if re_punc:
    x=x.translate(punct)
  return x

In [6]:
def apply_clean(batch):
  return {"text":[clean_text(x) for x in batch["text"]]}

In [7]:
dataset=dataset.map(apply_clean,batched=True)
print(dataset["train"][0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUSYELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967 I also heard that at first it was seized by US customs if it ever tried to enter this country therefore being a fan of films considered controversial I really had to see this for myself  The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States In between asking politicians and ordinary denizens of Stockholm about their opinions on politics she has sex with her drama teacher classmates and married men  What kills me about I AM CURIOUSYELLOW is that 40 years ago this was considered pornographic Really the sex and nudity scenes are few and far between even then its not shot like some

#Tokenize

In [8]:
model_name='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
max_len = 256

def tokenize(batch):
  return tokenizer(batch['text'], truncation=True, max_length=max_len)

In [10]:
tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])
print(tokenized["train"][0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0, 'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 6672, 7174, 2860, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 2149, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 6801, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 2016, 2038, 3348, 2007, 2014, 3689, 3836, 19846, 1998, 2496, 2273, 2054, 8563, 2033, 2055, 1045, 2572, 8025, 6672, 7174, 2860, 2003, 20

#Padding & Mask

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_loader = DataLoader(tokenized["train"], batch_size=8, collate_fn=data_collator, shuffle=True)

In [12]:
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape)

input_ids torch.Size([8, 256])
token_type_ids torch.Size([8, 256])
attention_mask torch.Size([8, 256])
labels torch.Size([8])


In [13]:
tokenized.save_to_disk("processed_dataset")
print("Dataset ready & saved!")

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset ready & saved!


#Load Model

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Metrics

In [15]:
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels),
        "f1": f1_metric.compute(predictions=preds, references=labels, average="weighted")
    }

#Training Setup

In [17]:
args = TrainingArguments(
    "bert-imdb",
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2518362573.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


#Training

In [18]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akhyar (akhyar-university-of-engineering-and-technology-lahore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.649200
100,0.376700
150,0.320000
200,0.300200
250,0.311400
300,0.322800
350,0.289000
400,0.238600
450,0.275400
500,0.283100


TrainOutput(global_step=3126, training_loss=0.21155082271866363, metrics={'train_runtime': 2232.7624, 'train_samples_per_second': 22.394, 'train_steps_per_second': 1.4, 'total_flos': 6576633495353280.0, 'train_loss': 0.21155082271866363, 'epoch': 2.0})

#Evaluation

In [19]:
metrics = trainer.evaluate()
print(metrics)

Trainer is attempting to log a value of "{'accuracy': 0.92576}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9257589052705135}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2666586637496948, 'eval_accuracy': {'accuracy': 0.92576}, 'eval_f1': {'f1': 0.9257589052705135}, 'eval_runtime': 339.2529, 'eval_samples_per_second': 73.691, 'eval_steps_per_second': 4.607, 'epoch': 2.0}


In [20]:
%load_ext tensorboard
%tensorboard --logdir ./logs

Output hidden; open in https://colab.research.google.com to view.